<a href="https://colab.research.google.com/github/rishabhgaurav007/Machine-Learning/blob/master/ImageSplicing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import files
!pip install -q kaggle
import os
os.environ['KAGGLE_USERNAME'] = "gurudeepkuwar"
os.environ['KAGGLE_KEY'] = "0c7a28418c8d3e28bf34b04cc6af54f8"

In [2]:
!kaggle datasets download -d sophatvathana/casia-dataset

100% 5.21G/5.22G [01:12<00:00, 59.6MB/s]
100% 5.22G/5.22G [01:12<00:00, 77.2MB/s]


In [0]:
!unzip casia-dataset.zip

In [0]:
#imports here
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [0]:
import os
path_orig = 'casia/CASIA2/Au/' #Authentic 
path_modif = 'casia/CASIA2/Tp/' #Tampered

folder_orig = os.listdir()
folder_modif = os.listdir()

strings = []

for file in os.listdir(path_orig):
    if file.endswith('jpg'):
      if int(os.stat(path_orig + file).st_size) > 10000:
        line =  path_orig + file  + ',1\n'
        strings.append(line)

for file in os.listdir(path_modif):
      if file.endswith('jpg') or file.endswith('tif'):
         if int(os.stat(path_modif + file).st_size) > 10000:
            line =  path_modif + file + ',0\n'
            strings.append(line)

#run once to create the file
for line in strings:
      with open('casia/dataset.csv', 'a') as f:
         f.write(line)

In [6]:
#reading csv
df = pd.read_csv('casia/dataset.csv',names=['Path','Class'])
df

,Path,Class
0,casia/CASIA2/Au/Au_ani_30325.jpg,1
1,casia/CASIA2/Au/Au_arc_30059.jpg,1
2,casia/CASIA2/Au/Au_sec_20079.jpg,1
3,casia/CASIA2/Au/Au_art_30483.jpg,1
4,casia/CASIA2/Au/Au_sec_30259.jpg,1
...,...,...
12384,casia/CASIA2/Tp/Tp_S_NND_M_N_nat00023_nat00023...,0
12385,casia/CASIA2/Tp/Tp_D_CNN_M_N_arc00086_arc00086...,0
12386,casia/CASIA2/Tp/Tp_S_CRD_S_N_sec00020_sec00020...,0
12387,casia/CASIA2/Tp/Tp_S_NRN_S_N_ind00040_ind00040...,0


In [0]:
#Feature Extraction Using GLCM
from skimage.feature import greycomatrix,greycoprops
import cv2
from sklearn import preprocessing

In [0]:
properties=["correlation","contrast","homogeneity","energy","dissimilarity"]

DISTANCES = [1,2,3]
ANGLES = [0.,np.pi/4.,np.pi/2.,3.*np.pi/4.]

data = []
cou=0;
for row in df.itertuples():
  print(cou);
  cou=cou+1;
  img = cv2.imread(row.Path)
  imgYCB=cv2.cvtColor(img,cv2.COLOR_BGR2YCrCb)
  img_arr = np.array(imgYCB)
  imgans = []
  for i in img_arr:
    ans=[]
    for j in i:
      ans.append(j[1]) #Cb only
    imgans.append(ans)
  imgans = np.array(imgans)
  lf=[]
  for dis in DISTANCES:
    for rad in ANGLES:
      glcm = greycomatrix(imgans,distances=[dis],angles=[rad],levels=256,symmetric=True,normed=True)
      for k in properties:
        lf.append(greycoprops(glcm,k)[0,0])
  data.append(np.array(lf))
data = np.array(data)


In [0]:
X_glcm = data
X=preprocessing.normalize(X_glcm,norm="l1")
y = df['Class'].to_numpy()

In [59]:
X.shape


(12389, 60)

In [0]:
import random
random.shuffle(X)

In [60]:
y.shape

(12389,)

In [61]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)
X_train.shape

(8672, 60)

In [62]:
from sklearn.svm import SVC
clf = SVC(gamma='auto')
clf.fit(X_train,y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [63]:
clf.score(X_test,y_test)

0.5873015873015873

In [0]:
##

from numpy import savetxt
savetxt('glcm60.csv',X,delimiter = ',')

**done**

In [66]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
logreg = LogisticRegression()
logreg.fit(X_train, y_train)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [67]:
logreg.score(X_test,y_test)

0.7688996502555825

In [0]:
##


In [69]:
from sklearn.ensemble import RandomForestClassifier
acc=[]
n=40
for k in range(4,11):
    classifier = RandomForestClassifier(n_estimators=(k*10),criterion = 'entropy',random_state=0)
    classifier.fit(X_train,y_train)
    y_pred=classifier.predict(X_test)
    acc.append(classifier.score(X_test,y_test))
acc

[0.9410815173527038,
 0.9413505515200431,
 0.9397363465160076,
 0.9421576540220608,
 0.9416195856873822,
 0.9410815173527038,
 0.9413505515200431]

In [0]:
#

casia  CASIA1  CASIA2  casia-dataset.zip  drive  glcm60.csv  sample_data


In [126]:
from numpy import genfromtxt
X2= genfromtxt('/content/drive/My Drive/LBP_feature.csv', delimiter=',')
X2.shape

(12389, 162)

In [114]:
X.shape

(12389, 162)

In [93]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
logreg.score(X_test,y_test)

0.6298089857411892

In [95]:
from sklearn.ensemble import RandomForestClassifier
acc=[]
for k in range(4,11):
    classifier = RandomForestClassifier(n_estimators=(k*10),criterion = 'entropy',random_state=0)
    classifier.fit(X_train,y_train)
    y_pred=classifier.predict(X_test)
    acc.append(classifier.score(X_test,y_test))
acc

[0.6674737691686844,
 0.6704331450094162,
 0.6704331450094162,
 0.6698950766747377,
 0.6696260425073984,
 0.6698950766747377,
 0.6725854183481302]

In [0]:
from numpy import genfromtxt
X= genfromtxt('/content/glcm60.csv', delimiter=',')

In [125]:
X.shape

(12389, 60)

In [0]:
X=np.concatenate((X,X2),axis=1)

In [129]:
X.shape

(12389, 222)

In [0]:
X=preprocessing.normalize(X,norm="l1")


In [131]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
logreg.score(X_test,y_test)

0.5897228948076406

In [132]:
from sklearn.ensemble import RandomForestClassifier
acc=[]
for k in range(4,11):
    classifier = RandomForestClassifier(n_estimators=(k*10),criterion = 'entropy',random_state=0)
    classifier.fit(X_train,y_train)
    y_pred=classifier.predict(X_test)
    acc.append(classifier.score(X_test,y_test))
acc

[0.9192897497982244,
 0.9206349206349206,
 0.9200968523002422,
 0.9187516814635459,
 0.9198278181329029,
 0.9195587839655637,
 0.9184826472962067]

In [134]:
from sklearn.svm import SVC
clf = SVC(gamma='auto')
clf.fit(X_train,y_train)
clf.predict(X_test)
clf.score(X_test,y_test)

0.5873015873015873